## NODES

## Example 1

original code
```
sourcescrub_persons_df = cursor.execute(
    'SELECT ID, FULL_NAME, CITY, STATE, COUNTRY FROM L2.GRAPH_SS_PERSON').fetch_pandas_all()

sourcescrub_persons_df.rename(
    columns={'ID': 'SOURCE_ID', 'FULL_NAME': 'NAME'},
    inplace=True)

sourcescrub_persons = np.array_split(
    [{k: v for k, v in x.items() if v == v} for x in sourcescrub_persons_df.to_dict('records')], BATCHES)

# code to create the person type node
for chunk_df in sourcescrub_persons:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("Person", "SourcescrubPerson"), "SOURCE_ID"),
    )
```


new code to define node:
node = {
    "source": "graph_ss_person",
    "labels": ("Person", "Sourcescrub"),
    "properties_list": [
        {
            "property": "source_id",
            "field": "ID",
            "type": "string",
        },
        {
            "property": "name",
            "field": "full_name",
            "type": "string",
        },
        {
            "property": "city",
            "field": "city",
            "type": "string",
        },
        {
            "property": "state",
            "field": "state",
            "type": "string",
        },
        {
            "property": "country",
            "field": "country",
            "type": "string",
        },
    ],
    # based on th statememnt
    "merge_key": (("Person"), {"property": "source_id", "field": "ID"}),
    "batch_size_factor": 1.0,
}
NODES.append(node)

In [ ]:
NODES = []

"""
Organization nodes
"""

node = {
    "source": "ss_org",
    "labels": ("Organization", "Sourcescrub"),
    "properties_list": [
        {
            "property": "source_id",
            "field": "ID",
            "type": "string",
        },
        {
            "property": "",
            "field": "",
            "type": "",
        }
    ],
    "merge_key": (("Organization",), {"property": "source_id", "field": "ID"}),
    "batch_size_factor": 1.0,
}
NODES.append(node)

node = {
    "source": "cb_org",
    "labels": ("Organization", "Crunchbase"),
    "properties_list": [
        {
            "property": "source_id",
            "field": "crunchbase_organization_uuid",
            "type": "string",
        },
        {
            "property": "",
            "field": "",
            "type": "",
        }
    ],
    "merge_key": (("Organization",), {"property": "source_id", "field": "crunchbase_organization_uuid"}),
    "batch_size_factor": 1.0,
}
NODES.append(node)

node = {
    "source": "ss_organizational_investor",
    "labels": ("Organization", "Sourcescrub", "Investor"),
    "properties_list": [
        {
            "property": "source_id",
            "field": "ID",
            "type": "integer",
        },
        {
            "property": "",
            "field": "",
            "type": "",
        }
    ],
    "merge_key": (("Organization",), {"property": "source_id", "field": "ID"}),
    "batch_size_factor": 1.0,
}
NODES.append(node)

node = {
    "source": "cb_organizational_investor",
    "labels": ("Organization", "Crunchbase", "Investor"),
    "properties_list": [
        {
            "property": "source_id",
            "field": "crunchbase_organization_uuid",
            "type": "string",
        },
        {
            "property": "",
            "field": "",
            "type": "",
        }
    ],
    "merge_key": (("Organization",), {"property": "source_id", "field": "crunchbase_organization_uuid"}),
    "batch_size_factor": 1.0,
}
NODES.append(node)

## RELATIONSHIPS

In [ ]:

RELATIONSHIPS = []

"""
INVESTED_IN (keep relationship types agnostic to data source)
"""

relationship = {
    "source": "ss_organizational_investor_to_org",
    "properties_list": [],
    "merge_key": ("INVESTED_IN",),
    "start_node_key": (("Organization",), {"property": "source_id", "field": "investor_id"}),
    "end_node_key": (("Organization",), {"property": "source_id", "field": "company_id"}),
    "batch_size_factor": 1.0,
}
RELATIONSHIPS.append(relationship)

relationship = {
    "source": "cb_organizational_investor_to_org",
    "properties_list": [],
    "merge_key": ("INVESTED_IN",),
    "start_node_key": (("Organization",), {"property": "source_id", "field": "investor_id"}),
    "end_node_key": (("Organization",), {"property": "source_id", "field": "company_id"}),
    "batch_size_factor": 1.0,
}
RELATIONSHIPS.append(relationship)

## CONSTRAINTS

In [ ]:

# CONSTRAINTS = [
#     """
#     CREATE CONSTRAINT unique_org IF NOT EXISTS
#     FOR (c:Organization) REQUIRE c.source_id IS UNIQUE
#     """,
#     """
#     CREATE CONSTRAINT unique_url IF NOT EXISTS
#     FOR (u:Url) REQUIRE u.url IS UNIQUE
#     """,
#
# ]

# person be unique
CONSTRAINTS = [
    """
    CREATE CONSTRAINT unique_person IF NOT EXISTS 
    FOR (c:Person) REQUIRE c.source_id IS UNIQUE 
    """,
    """
    CREATE CONSTRAINT unique_url IF NOT EXISTS 
    FOR (u:Url) REQUIRE u.url IS UNIQUE 
    """
]